## 웹 크롤링

### 인터넷 접속 라이브러리 추가

- urllib.request 모듈로 작업
- html 소스 로드로 별다른 문제 없음


In [91]:
from urllib.request import urlopen, Request # 5일차 참고

# 도시별 날씨 검색함수
def get_weather(city):
    # 기상청 웹 사이트 도시별 날씨 페이지
    url = 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
    page = urlopen(url=url)

    text = page.read().decode('utf-8')
    text = text[text.find(f'>{city}</a>'):]
    
    # 부산의 현재 기온 가져오기
    for i in range(5): 
        text = text[text.find('<td>')+1:]

    start = 3
    end = text.find('</td>') # td>7.1</td> ......td>(문자가 3개여서 start = 3) ......</td>만나는 지점이 끝나는 지점
    current_temp = text[start:end]
    print(f'{city}의 현재 기온은 {current_temp}˚C 입니다.')

    # 습도 가져오기
    for i in range(5): # 5로 받았기 때문에 10번째를 가져오려면 5으로 해줘야함
        text = text[text.find('<td>')+1:]

    start = 3
    end = text.find('</td>')
    current_humid = text[start:end]
    print(f'{city}의 현재 습도는 {current_humid}% 입니다.')



get_weather('부산')

부산의 현재 기온은 12.9˚C 입니다.
부산의 현재 습도는 23% 입니다.


### OpenAPI 크롤링

- 공공데이토 포털
    - https://www.data.go.kr
    - 부산광역시 버스정보안내기 현황

- 문제점
    ```python
    responce = requests.get(total_url, verify = False)
    ```
    - https를 일반 request로 부르면 SSL 오류발생
    - 해결하려면 외부모듈 requests를 사용, verify=False 옵션을 지정
    - 아래 셀의 19번 라인!

In [92]:
import requests
from urllib.parse import quote, unquote, urlencode # 한글을 url encode로 변환하는 함수  -----> 롯데 : %EB%A1%AF%EB%8D%B0
import json
import ssl

# stationName : 정류소 이름
def getDataPortalSearch(stationName, type):
    api_url = 'https://apis.data.go.kr/6260000/BusanTblBusinfoeqStusService/getTblBusinfoeqStusInfo'
    queryString = "?" + urlencode(
        {
            'servicekey' : 'COBU+264cBDGC1gBxcKNfV8/PYElrUbY2f2+es3jW9MoAL4ILj2qlUIasR4gVtO4mflZnqTrEv7EXqjL/tcq5g==',
            'pageNo' : '1',
            'numOfRows' : '10',
            'resultType' : type,
            'stationLoc' : stationName
        }
    )
    total_url = api_url + queryString
    responce = requests.get(total_url, verify = False)
    return responce.text
try:
    result = getDataPortalSearch('대학교','json')
    json_data = json.loads(result)
    station_data = json_data['getTblBusinfoeqStusInfo']['body']['items']['item']
    #[{'stationNum': '09283', 'stationLoc': '롯데백화점센텀시티점', 'lat': '35.16967324', 'lng': '129.1317843', 'addr': '해운대구 우동 1496', 'insYear': '', 'dataDay': '2023-02-03'}, 
    # {'stationNum': '09336', 'stationLoc': '롯데백화점센텀시티점', 'lat': '35.16997953', 'lng': '129.1320357', 'addr': '해운대구 우동 1498', 'insYear': '', 'dataDay': '2023-02-03'}, 
    # {'stationNum': '06030', 'stationLoc': '롯데백화점동래점.명륜역', 'lat': '35.21276314', 'lng': '129.0781263', 'addr': '동래구 온천1동 474-25', 'insYear': '', 'dataDay': '2023-02-03'}, 
    # {'stationNum': '06046', 'stationLoc': '롯데백화점동래점', 'lat': '35.2101971', 'lng': '129.0778307', 'addr': '동래구 온천2동 503-46', 'insYear': '', 'dataDay': '2023-02-03'}]

    for i in station_data:
        print(i)
except Exception as e:
    print('찾는 데이터가 없습니다')


{'stationNum': '15056', 'stationLoc': '신라대학교입구', 'lat': '35.17011792', 'lng': '128.9886419', 'addr': '사상구 덕포1동 788-8', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '15261', 'stationLoc': '신라대학교', 'lat': '35.16698667', 'lng': '128.9950633', 'addr': '사상구 괘법동 208-2', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '13072', 'stationLoc': '부산경상대학교서문', 'lat': '35.18558167', 'lng': '129.0977667', 'addr': '연제구 연산동 1092-3', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '13157', 'stationLoc': '부산경상대학교서문', 'lat': '35.18574896', 'lng': '129.0974727', 'addr': '연제구 연산동 374-31', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '11080', 'stationLoc': '부산대학교정문', 'lat': '35.23236166', 'lng': '129.0869587', 'addr': '금정구 장전3동 416-27', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '11085', 'stationLoc': '부산대학교정문', 'lat': '35.23193029', 'lng': '129.0851576', 'addr': '금정구 장전2동 419-45', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '11088', 'stationLoc': '부산대학

c:\DEV\Langs\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


### OpenAPI로 가져온 데이터 지도 표시

- 문제점
    - 딕셔너리 구조하고 문자열 포맷팅 f'{}'호환안됨
    - 구식방법 문자열 결합방식으로 해결
    ```python
    stop_str = '<h4>' + item['stationLoc'] + '</h4>' + item['stationNum'] + '<br>' + item['addr']
    ```

In [93]:
!pip install folium

In [94]:
import folium
if len(station_data) > 0:
    center_lat = station_data[0]['lat']
    center_lng = station_data[0]['lng']

m = folium.Map(location=[center_lat,center_lng], zoom_start = 13)

# 전체정류소 위치값 마커 표시
for item in station_data:
    stop_str = '<h4>' + item['stationLoc'] + '</h4>' + item['stationNum'] + '<br>' + item['addr']
    Iframe = folium.IFrame(stop_str)
    popup = folium.Popup(Iframe,min_width = 200, max_width = 200) #팝업사이즈 지정
    folium.Marker(location=[item['lat'], item['lng']], popup=popup,
                  icon=folium.Icon(icon='pushpin')).add_to(m)

m

### beautyfulsoup (version 4)
[참고1](https://wikidocs.net/85739)
[참고2](https://wikidocs.net/86334)

웹 크롤링을 편하게 해주는 도구(라이브러리)
```shell
pip install beautifulsoup4
```

In [95]:
!pip install beautifulsoup4

In [96]:
from bs4 import BeautifulSoup
import requests

url = 'https://kin.naver.com/search/list.naver?query=%EC%A0%9C%EC%9E%84%EC%8A%A4%EC%9B%B9+%EB%A7%9D%EC%9B%90%EA%B2%BD' #지식인 제임스웹 망원경

response = requests.get(url)
if response.status_code == 200: # 웹페이지 OK
    html = response.text
    soup = BeautifulSoup(html,'html.parser') 
    ul = soup.select_one('ul.basic1')
    titles = ul.select('li > dl > dt > a')
    for title in titles:
        print(title.get_text())
    # print(soup)
    # title = soup.select_one('#s_content > div.section > ul > li:nth-child(1) > dl > dt > a')
    # print(title)
    # print(title.get_text()) # 타이틀만 가져온다
else:
    print(f'Error : {response.status_code}')


제임스웹 우주망원경 질문이요
최고고도 부탁드립니다. 제임스웹 망원경이
제임스 웹 망원경 우주사진 원본 어디서... 
제임스 웹 망원경 촬영 원리
제임스 웹 망원경
제임스 웹 우주망원경이 찍은 첫 사진은... 
제임스 웹 망원경의 종류 알려주세요
제임스 웹 우주망원경
제임스웹 망원경
제임스 웹 우주 망원경 제작 기간이
